## Present Value of Liabilities and Funding Ratio

Goal is to answer one of the most important purpose of investing.

The goal of investing is not simply to grow your assets to as large as possible. The goal of investing is to make sure that you will have the money that you need to do the things that you want to do in the future.

We are going to come up with some basic functionality that allows us to measure two things:

    1. What is future liability / future consumption? How to model that?

    2. Do I have enough money today to be able to meet that, and how do I measure that? (Funding ratio)


Present value of a set of liabilities L where each liability $L_i$ is due at time $t_i$ is given by:
$ PV(L) = \sum\limits _{i=1}^{k}B(t_i)L_i $ 

where $B(t_i)$ is the price of a pure discount bond that pays 1 dollar at time, $t_i$ as you have to discount them somehow for the time value of money. In other words, you discount each liability. So you discount each liability by a factor, what would $1, at time $t_i$ in the future, cost me today to buy? That is the price of a pure discount bond.

If we assume the yield curve is lat and the annual rate of interest is $r$ then $B(t)$ is given by 

$$B(t) = \frac{1}{(1+r)^t}$$

or 

$$B(t)(1+r)^t = 1$$

B(t) is simply the amount of money such that, when compounded at the current interest rate t times, will give $1.

In [1]:
import pandas as pd
import numpy as np
import risk_kit as kit
%load_ext autoreload
%autoreload 2

In [2]:
def discount(t, r):
    """
    Compute the price of a pure discount bond that pays a dollar at time t, given interest rate r
    """
    return (1+r)**(-t)

In [4]:
# Assume that the yield curve is flat. That means that the interest rate is the same, regardless of whether it is 1,3 or 5 years.
discount(10, .03)

0.7440939148967249

It takes 75cents to get 1$ back at the end of 10 years. We can verify this by compounding.

In [6]:
0.7440939148967249*(1.03**10)

1.0

Now that we know how to discount a liability, we can extend that to computing the present value. The present value is nothing more than the sum of the discounted liabilities.

In [8]:
# Present value of set of liabilities given an interest rate r
def pv(l, r):
    """
    Computes the present value of a sequence of liabilities
    l is indexed by the time, and the values are the amounts of each liability returns the present value of the sequence.
    """
    
    dates = l.index
    discounts = discount(dates, r)
    return (discounts*l).sum()
    

In [10]:
# Assume liabilities 1million, 1.5, 2, 2.5 millions to pay in 3 , 3.5, 4, 4.5 years resp.
liabilities = pd.Series(data=[1, 1.5, 2, 2.5], index=[3, 3.5, 4, 4.5] )
liabilities

3.0    1.0
3.5    1.5
4.0    2.0
4.5    2.5
dtype: float64

In [11]:
# Assume interest flat rate 3%
pv(liabilities, 0.03)

6.233320315080045

In [12]:
liabilities.sum()

7.0

What this means is the present value of liabilities because of time value of money is 6.2million $. So, in the future you owe 7million$. If you had 6.2million today and you have this particular sequence of liabilities in the future, you can pay it off.

This is defined by Funding ratio

In [13]:
def funding_ratio(assets, liabilities, r):
    """
    Computes the funding ratio of some assets given liabilities and interest rate
    """
    return assets/pv(liabilities, r)

In [14]:
# Assume you have 5million $
funding_ratio(5, liabilities, 0.03)

0.8021407126958777

This means that you are 80% funded. That means, unless you do something else, you cannot put your 5million dollars in bonds and expect to meet your liabilities.

In [15]:
# Assume that the interest rates go down
funding_ratio(5, liabilities, 0.02)

0.7720304366941648

The funding ratio is now worse. The reason is that the value of the liabilities have gone up. In other words, the amount by which the assets that you have now will grow more slowly and so you won't be able to meet your liabilities.

In [16]:
# Assume that the interest rates go up
funding_ratio(5, liabilities, 0.05)

0.8649082768407927

Now the funding ratio has gone up

In [19]:
import ipywidgets as widgets
from IPython.display import display
%matplotlib inline

def show_funding_ratio(assets, r):
    """
    How does the funding ratio change when either the assets I have increase or the interest charge increase.
    """
    fr = funding_ratio(assets, liabilities, r)
    print(f'{fr*100:.2f}')
    
controls = widgets.interactive(show_funding_ratio,
                              assets = widgets.IntSlider(min=1, max=10, step=1, value=5),
                               r = (0,.20, .01)
                              )
display(controls)

interactive(children=(IntSlider(value=5, description='assets', max=10, min=1), FloatSlider(value=0.1, descript…

When assets increase, funding ratio increases. Around 5 million and 10% interest rate, you have just enough to cover the liabilities. If rate decreases, the funding ratio decreases.